# Скачаем данные и установим необходимые программы

In [ ]:
!wget https://raw.githubusercontent.com/knapweedss/hse21_hw2/main/data/gms2.lst
!wget https://raw.githubusercontent.com/knapweedss/hse21_hw2/main/data/scaffolds.fa
!wget https://raw.githubusercontent.com/knapweedss/hse21_hw2/main/data/proteins.fasta
!wget https://raw.githubusercontent.com/knapweedss/hse21_hw2/main/data/scaffolds.hits_from_MIL_1.txt
!wget https://raw.githubusercontent.com/knapweedss/hse21_hw2/main/data/scaffolds.hits_from_SwissProt.txt

--2021-12-19 20:23:46--  https://raw.githubusercontent.com/knapweedss/hse21_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221547 (216K) [text/plain]
Saving to: ‘gms2.lst.1’

gms2.lst.1          100%[===================>] 216.35K  --.-KB/s    in 0.02s   

2021-12-19 20:23:46 (8.51 MB/s) - ‘gms2.lst.1’ saved [221547/221547]

--2021-12-19 20:23:47--  https://raw.githubusercontent.com/knapweedss/hse21_hw2/main/data/scaffolds.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3967630 (3.8M) [text/plain]
Saving 

## Скачиваем геном близкородственной бактерии T.oleivorans

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

- Устанавливаем биопитон

In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 5.1 MB/s 


In [ ]:
!head -n 20 gms2.lst

# GeneMark.hmm-2 LST format
# GeneMark.hmm-2 prokaryotic version: 1.25_lic
# File with sequence: scaffolds.fasta
# File with native parameters: GMS2.mod
# Native species name and build: unspecified GeneMarkS-2-1.14_1.25_lic
# File with MetaGeneMark parameters: /content/gms2_linux_64/mgm_11.mod
# translation table: 11
# output date start: Sun Dec 19 00:58:14 2021

# sequence-region 1 3875869
SequenceID: scaffold1_cov231
     1   -   <3    299     297 native GAGGTG 4 1
     2   -    372    815     444 atypical AGCGAG 6 1
     3   -    840    1361     522 atypical CAGGAA 7 1
     4   -    1799    2626     828 atypical CAGGAT 10 1
     5   +    2972    3094     123 native TTGGAG 7 1
     6   -    3269    4003     735 native CAGGTC 8 1
     7   -    4341    5516    1176 native TAGGAG 9 1
     8   -    5541    7688    2148 native GTGGAG 8 1
     9   +    8057    8851     795 native AAGTAG 6 1


- найдем длину самого длинного скаффолда (по предыдущему дз cov231)

# SeqIO

In [ ]:
# для удобства из библиотеки
#record = SeqIO.read("Fasta/f001", "fasta")
#record = next(SeqIO.parse("Fasta/f002", "fasta"))
#for record in SeqIO.parse("Fasta/f002", "fasta")
#record_dict = SeqIO.to_dict(SeqIO.parse("Fasta/f002", "fasta"))
#SeqIO.write(records, "example.faa", "fasta"
#from Bio.SeqFeature import SeqFeature, FeatureLocation
#f1 = SeqFeature(FeatureLocation(5, 10), type="domain")
#f1.strand == f1.location.strand == None
#True

In [ ]:
from Bio import SeqIO

records = []
num = []

for j in SeqIO.parse("/content/scaffolds.fa", "fasta"):
  records.append(j)
  num.append(j.id[j.id.find('cov')+3:])
  if j.id == 'scaffold1_cov231':   
    record_longest = j

In [ ]:
print(len(records)) # количество скаффолдов

72


In [ ]:
print(len(record_longest)) # длина самого длинного скаффолда

3875869


# Аннотация генома

- импортируем необходимые модули

In [ ]:
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

- создаем словарь 

In [ ]:
scaffolds = dict()
for record in SeqIO.parse("scaffolds.fa", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['organism'] = 'Thalassolituus oleivorans Barents'
  record.annotations['source'] = 'Thalassolituus oleivorans Barents'
  record.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  record.annotations['data_file_division'] = 'BCT'
  
  scaffolds[record.id] = record

In [ ]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  desc = gene.description.split(' ')
  scaffold = desc[1]
  start, end = int(desc[2]), int(desc[3])
  strand = 1 if desc[4] == '+' else -1
  
  feat = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [desc[0]]
  feat.qualifiers['translation'] = [gene.seq]
  scaffolds[scaffold].features.append(feat)
  
  genes[desc[0]] = feat

- добавляем функции белков

In [ ]:
mil = dict()
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in feat.qualifiers:
    continue
  if 'product' not in feat.qualifiers:
    continue
  
  mil[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

In [ ]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mmatch = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
match = mmatch[mmatch['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
#print(mmatch)

In [ ]:
#print(match)

In [ ]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

File ‘uniprot_sprot.dat.gz’ already there; not retrieving.

gzip: uniprot_sprot.dat already exists; do you wish to overwrite (y or n)? т
	not overwritten


In [ ]:
for i, hit in match.iterrows():
  gene = genes[str(hit['qseqid'])]
  m = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil[m]]

- SwissProt

In [ ]:
# чтобы не обрабатывать большой файл легче достать новое и перезаписать
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SP.txt 

In [ ]:
previd = None
swissgenes = dict()
for line in open('SP.txt'):
  if line.startswith('ID'):
    previd = line.split()[1]
  if line.startswith('DE'):
    swissgenes[previd] = line.split('=')[1][:-2]

In [ ]:
swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swissgenes[match]]

- запись в GENOME.gkb

In [ ]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

72